In [87]:
!pip install -U scikit-learn

     |████████████████████████████████| 7.2 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 302 kB 31.4 MB/s eta 0:00:01
     |████████████████████████████████| 30.7 MB 2.1 MB/s eta 0:00:01    |█▎                              | 1.2 MB 2.1 MB/s eta 0:00:14     |██▌                             | 2.4 MB 2.1 MB/s eta 0:00:14     |███████████████████████         | 22.0 MB 2.1 MB/s eta 0:00:05
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)


In [138]:
import pandas as pd
import numpy as np
import os
from functools import reduce
from us import states
from sklearn.linear_model import LinearRegression
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)

In [139]:
# Restaurants
res_list = [
    'whole', 'wendys', 'trader', 'ctown', 'aldi', 'bk_docs'
]

In [140]:
df_list = []

In [141]:
for r in res_list:
    check = [(i, i.startswith(r)) for i in os.listdir('merged_files/')]
    for c in check:
        if c[1]:
            df = pd.read_csv(f'merged_files/{c[0]}')
            name = df.iloc[1]['Company Name']
            df = df[df['Company Name'] == name]
            df = (
                df.groupby(['State','County', 'Company Name'], as_index=False)['ZIP Code']
                .count()
                .sort_values('ZIP Code',ascending=False)
                .rename(columns={'ZIP Code': f'{name} Count'})
                .reset_index(drop=True)
            ).drop('Company Name', axis=1)
            df_list.append(df)

In [142]:
restaurants_df = reduce(lambda  left,right: pd.merge(left,right,on=['State', 'County'],
                                            how='outer'), df_list)


In [143]:
restaurants_df['State'] = restaurants_df['State'].apply(lambda x: states.lookup(x).name)

In [159]:
restaurants_df[restaurants_df['state'] == "Florida"]

,state,county,Whole Foods Market Count,Wendy's Count,Trader Joe's Count,C-Town Count,ALDI Count,Burger King Count
1,Florida,Miami Dade County,9.0,60,3.0,NaN,14.0,79.0
2,Florida,Broward County,8.0,53,4.0,NaN,15.0,49.0
3,Florida,Palm Beach County,5.0,33,5.0,NaN,10.0,33.0


In [144]:
restaurants_df.rename(columns={
    'State': 'state',
    'County': 'county'
}, inplace=True)

In [145]:
pop_dem_income_df = pd.read_csv('merged_files/Pop-Demo-Income.csv')

In [146]:
restaurants_df['county'] = restaurants_df['county'] + " County" 

In [147]:
all_merged = pd.merge(pop_dem_income_df, restaurants_df, how='left', on=['state', 'county'])

In [150]:
all_merged[all_merged['state'] == "New York"].head()

,state,county,year,population,White,Black,American_Indian,Asian,Native_Hawaian,Other,White-Black,White-AI,White-Asian,Black-AI,All-other-two,Three,Four,B19019_001E,Whole Foods Market Count,Wendy's Count,Trader Joe's Count,C-Town Count,ALDI Count,Burger King Count
1860,New York,New York,2010,"19,399,878",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1861,New York,Albany County,2010,"304,086",239946.0,36817.0,716.0,13414.0,185.0,6504.0,2392.0,574.0,1337.0,496.0,1309.0,327.0,15.0,56090.0,NaN,NaN,NaN,NaN,NaN,NaN
1862,New York,Allegany County,2010,"48,971",47388.0,507.0,157.0,487.0,12.0,144.0,115.0,137.0,31.0,5.0,89.0,33.0,0.0,41305.0,NaN,NaN,NaN,NaN,NaN,NaN
1863,New York,Bronx County,2010,"1,387,298",299048.0,470130.0,6453.0,49076.0,177.0,497986.0,7264.0,2097.0,1618.0,1912.0,27721.0,2047.0,196.0,34264.0,NaN,8.0,NaN,22.0,3.0,18.0
1864,New York,Broome County,2010,"200,481",178141.0,8832.0,615.0,6659.0,20.0,2093.0,2149.0,621.0,1024.0,73.0,406.0,138.0,33.0,44457.0,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
NYFL_df = all_merged[all_merged['state'].isin(['New York', 'Florida'])].reset_index()

In [152]:
NYFL_df.rename(columns={'B19019_001E': "Median_Income"}, inplace = True)

In [153]:
NYFL_df.to_csv('NYFL.csv', index=False)

In [155]:
NYFL_df.head(100)

,index,state,county,year,population,White,Black,American_Indian,Asian,Native_Hawaian,Other,White-Black,White-AI,White-Asian,Black-AI,All-other-two,Three,Four,Median_Income,Whole Foods Market Count,Wendy's Count,Trader Joe's Count,C-Town Count,ALDI Count,Burger King Count
0,329,Florida,Florida,2010,"18,845,537",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,330,Florida,Alachua County,2010,"247,614",173051.0,48755.0,470.0,12910.0,200.0,3192.0,1948.0,940.0,1087.0,217.0,913.0,553.0,11.0,40644.0,NaN,NaN,NaN,NaN,NaN,NaN
2,331,Florida,Baker County,2010,"27,066",22287.0,3515.0,30.0,93.0,8.0,127.0,173.0,230.0,12.0,15.0,58.0,0.0,0.0,47276.0,NaN,NaN,NaN,NaN,NaN,NaN
3,332,Florida,Bay County,2010,"169,206",137959.0,17354.0,923.0,3784.0,186.0,919.0,1558.0,1488.0,757.0,354.0,1222.0,278.0,16.0,47770.0,NaN,NaN,NaN,NaN,NaN,NaN
4,333,Florida,Bradford County,2010,"28,536",22011.0,5860.0,34.0,117.0,184.0,73.0,142.0,46.0,51.0,27.0,186.0,19.0,0.0,41126.0,NaN,NaN,NaN,NaN,NaN,NaN
5,334,Florida,Brevard County,2010,"543,965",453634.0,52758.0,1947.0,11035.0,319.0,8928.0,3351.0,2136.0,2640.0,166.0,2791.0,751.0,127.0,49523.0,NaN,NaN,NaN,NaN,NaN,NaN
6,335,Florida,Broward County,2010,"1,752,803",1111680.0,448777.0,3235.0,55794.0,574.0,70324.0,13549.0,3945.0,5622.0,1148.0,16955.0,2384.0,152.0,51694.0,8.0,53.0,4.0,NaN,15.0,49.0
7,336,Florida,Calhoun County,2010,"14,647",11668.0,2157.0,174.0,19.0,0.0,92.0,54.0,125.0,37.0,10.0,36.0,0.0,0.0,31699.0,NaN,NaN,NaN,NaN,NaN,NaN
8,337,Florida,Charlotte County,2010,"159,875",143923.0,8641.0,426.0,1708.0,0.0,2069.0,1001.0,519.0,629.0,0.0,283.0,186.0,0.0,45037.0,NaN,NaN,NaN,NaN,NaN,NaN
9,338,Florida,Citrus County,2010,"141,173",131271.0,3942.0,606.0,1851.0,33.0,1026.0,529.0,778.0,223.0,24.0,358.0,35.0,10.0,37933.0,NaN,NaN,NaN,NaN,NaN,NaN
